In [9]:
import pandas as pd
import numpy as np
import random

## Допустим
###  У нас есть некий продукт с чатом поддержки. Клиенты пишут в него, чтобы найти решение возникающим проблемам. На вопросы клиентов отвечают операторы. Чтобы облегчить нагрузку на операторов мы подключили чатбота, который в состоянии ответить на самые распростараненные вопросы.

### Таким образом чат поддержки генерирует поток данных, которые сохраняются в нашей базе данных в виде таблиц. Пусть одна из них имеет следующий формат:

- id - уникальный идентификатор диалога
- date - дата поступления диалога (в формате dd-mm-yyyy)
- participants - участники диалога ('CUST_BOT', 'CUST_BOT_OPER', 'CUST_OPER')
- theme - тема диалога; последний вопрос заданный клиентом ('THEME1', 'THEME2', 'THEME3')
- rate - оценка диалога клиентом (по 5-ти бальной шкале)
- duration - длительность диалога (в секундах)
- oper_wait_time - время, которое потребовалось оператору, чтобы подключится к клиенту; время ожидания оператора (в секундах)
- all_mes_count - общее количество сообщений в диалоге (в штуках)
- cust_mes_count - количество сообщений клиента в диалоге (в штуках)
- bot_mes_count - количество сообщений бота в диалоге (в штуках)
- oper_mes_count - количество сообщений оператора в диалоге (в штуках)
- oper_name - имя оператора, включившегося в разговор (None, 'Игорь', 'Полнина', 'Гоша','Николай', 'Владимир', 'Маша', 'Света', 'Андрей')

## Генерация синтетичестких данных для презентации

### Выберем размер нашей выборки

In [10]:
sample_size = 30000

### Теперь сгенерируем датасет в указанном формате, обладающий следующими особенностями:

###### id:
 - значения id представлены в виде десятизначной строки чисел

In [11]:
ids = ['{:010}'.format(i) for i in range(sample_size)]

###### date: 
 - Пусть количество диалогов в выходные дни будет в целом на 25% больше чем в будние дни 

In [4]:
unique_dates = pd.date_range(start='2021-07-01',end='2021-07-30')
weekends = [day for day in unique_dates if day.weekday() in [5,6]]
weekdays = [day for day in unique_dates if day.weekday() not in [5,6]]

dates = sorted([*np.random.choice(weekends, int(sample_size*0.625), replace=True), 
                *np.random.choice(weekdays, int(sample_size*0.375), replace=True)])

In [5]:
summary = len([i for i in dates if i in weekends]) + len([i for i in dates if i in weekdays])

wends = len([i for i in dates if i in weekends])
wdays = len([i for i in dates if i in weekdays])

print(wends/summary, wdays/summary, wends/summary - wdays/summary)

0.625 0.375 0.25


In [8]:
550+550*0.25

687.5

In [6]:
pd.Series(dates).value_counts().reset_index().sort_values('index')

,index,0
22,2021-07-01,499
9,2021-07-02,550
2,2021-07-03,2352
3,2021-07-04,2330
24,2021-07-05,494
10,2021-07-06,541
15,2021-07-07,525
27,2021-07-08,473
18,2021-07-09,518
0,2021-07-10,2474


In [28]:
pd.Series([i for i in dates if i in weekdays]).value_counts()

2021-07-14    549
2021-07-27    544
2021-07-30    540
2021-07-26    527
2021-07-06    523
2021-07-09    521
2021-07-07    520
2021-07-28    516
2021-07-20    514
2021-07-21    513
2021-07-08    510
2021-07-13    509
2021-07-15    509
2021-07-22    504
2021-07-16    504
2021-07-02    504
2021-07-12    503
2021-07-29    500
2021-07-19    495
2021-07-23    486
2021-07-01    483
2021-07-05    476
dtype: int64

###### participants: 
- Пусть значения колонки participants распределяются в следующем соотношении: CUST_BOT - 60%, CUST_BOT_OPER - 30%, CUST_OPER - 10%

In [5]:
participants = [
    *['CUST_BOT']*int(sample_size*0.6), 
    *['CUST_BOT_OPER']*int(sample_size*0.3), 
    *['CUST_OPER']*int(sample_size*0.1)
]
random.shuffle(participants)

###### oper_name
 - Бот закрывает 60% диалогов (У 60% диалогов oper_name == None)
 - Диалоги распределены между операторами случайным образом

In [6]:
unique_oper_names = ['Игорь', 'Полина', 'Гоша','Николай', 'Владимир', 'Маша', 'Света', 'Андрей']
oper_names = [None if part == 'CUST_BOT' else np.random.choice(unique_oper_names, 1, replace=True)[0] for part in participants]

###### theme 
 - Тематики распределены между диалогами случайным образом

In [7]:
unique_themes = ['THEME1', 'THEME2', 'THEME3']
themes = list(np.random.choice(unique_themes, sample_size, replace=True))

###### rates: 
- Пусть только 6% от всех диалогов были оценены клиентами
- Пусть клиенты оценивают диалоги в следующей пропорции: '1' - 30%, '2' - 1%, '3' - 2%, '4' - 2%, '5' - 65% 
- Пусть диалоги с тематикой THEME3 оцениваются 1-ей на 20% чаще (и 5-кой на 20% реже)
- Пусть диалоги в которых участвовал оператор Андрей оцениваются 1-ей на 10% чаще (и 5-кой на 10% реже)

In [8]:
rates = []
rate_probability = 0.06
probability_dict = {None: 1-rate_probability, 
                    '1': rate_probability*0.3, 
                    '2': rate_probability*0.01, 
                    '3': rate_probability*0.02, 
                    '4': rate_probability*0.02, 
                    '5': rate_probability*0.65}

for theme, name in zip(themes, oper_names):
    d = probability_dict.copy()
    
    if theme == 'THEME3':
        d['1'] += rate_probability*0.2
        d['5'] -= rate_probability*0.2
    if name == 'Андрей':
        d['1'] += rate_probability*0.1
        d['5'] -= rate_probability*0.1
    
    rates.append(np.random.choice(list(d.keys()), 1, p=list(d.values()))[0])


###### oper_wait_time:
 - Пусть минимальное время реакции оператора - 3 секунды, максимальная - 10 минут
 - Оператор Андрей включается в диалог на 5 минут позже чем остальные операторы

In [9]:
min_owt = 3
max_owt = 10*60

oper_wait_times = []

for part, oper_name in zip(participants, oper_names):
    if part in ['CUST_BOT_OPER', 'CUST_OPER']:
        if oper_name == 'Андрей':
            oper_wait_times.append(np.random.choice(range(min_owt+5*60+1, max_owt+5*60+1), 1)[0])
        else:
            oper_wait_times.append(np.random.choice(range(min_owt, max_owt), 1)[0])
    else:
        oper_wait_times.append(0)

###### duration:
- Длительность диалога не может быть меньше времени реакции оператора
- Пусть длительность диалога без учета времени ожидания оператора лежит в диапазоне от 10 секунд до 15 минут

In [10]:
min_addit_dur = 10
max_addit_dur = 15*60

durations = []

for wait_time in oper_wait_times:
    durations.append(np.random.choice(range(wait_time + min_addit_dur+1, wait_time + max_addit_dur+1), 1)[0])

###### cust_mes_count:
 - Пусть минимальное количество сообщений клиента - 1, максимальное - 25
 - Если в диалоге участвовал и бот и оператор (participants == 'CUST_BOT_OPER'), то количество сообщений клиента должно быть больше 1

In [11]:
min_cmc = 1
max_cmc = 25

cust_mes_counts = []

for part in participants:
    if part == 'CUST_BOT_OPER':
        if min_cmc <= 1:
            cust_mes_counts.append(np.random.choice(range(2, max_cmc), 1)[0])
    else:
        cust_mes_counts.append(np.random.choice(range(min_cmc, max_cmc), 1)[0])

###### bot_mes_count:
 - Пусть минимальное количество сообщений бота - 1
 - Если бот не участвовал в диалоге (participants == 'CUST_OPER'), то количество сообщений бота равно нулю (bot_mes_count == 0)
 - Если в диалоге участвовал только бот и клиент (participants == 'CUST_BOT'), то количество сообщений бота равно количеству сообщений клиента
 - Если в диалоге участвовал клиент, бот и оператор, (participants == 'CUST_BOT_OPER'), то количесво сообщений бота строго меньше количества сообщений клиента
 

In [12]:
min_bmc = 1
bot_mes_counts = []

for cmc, part in zip(cust_mes_counts, participants):
    if part == 'CUST_OPER':
        bot_mes_counts.append(0)
    elif part == 'CUST_BOT':
        bot_mes_counts.append(cmc)
    elif part == 'CUST_BOT_OPER':
        bot_mes_counts.append(np.random.choice(range(min_bmc, cmc), 1)[0])


###### oper_mes_count:
 - Пусть минимальное количество сообщений клиента - 1, максимальное - 35
 - Если оператор не участвовал в диалоге (participants == 'CUST_BOT'), то количество сообщений оператора равно нулю (oper_mes_count == 0)

In [13]:
min_omc = 1
max_omc = 35

oper_mes_counts = []

for part in participants:
    if part == 'CUST_BOT':
        oper_mes_counts.append(0)
    else:
        oper_mes_counts.append(np.random.choice(range(min_omc, max_omc), 1)[0])

###### all_mes_count:
 - Общее количество сообщений в диалоге это сумма сообщений клиента, бота и оператора

In [14]:
all_mes_counts = [sum(x) for x in zip(cust_mes_counts, bot_mes_counts, oper_mes_counts)]

### Формирует и сохраняем наш датасет

In [15]:
df = pd.DataFrame({
    'date': dates,
    'id': ids,
    'participants': participants, 
    'theme': themes,
    'rate': rates, 
    'oper_name': oper_names,
    'duration': durations, 
    'oper_wait_time': oper_wait_times,
    'all_mes_count': all_mes_counts, 
    'cust_mes_count': cust_mes_counts, 
    'bot_mes_count': bot_mes_counts, 
    'oper_mes_count': oper_mes_counts,
    
})

df.head()

,date,id,participants,theme,rate,oper_name,duration,oper_wait_time,all_mes_count,cust_mes_count,bot_mes_count,oper_mes_count
0,2021-07-01,0000000000,CUST_BOT,THEME2,None,None,257,0,12,6,6,0
1,2021-07-01,0000000001,CUST_BOT_OPER,THEME3,None,Полина,414,337,46,9,5,32
2,2021-07-01,0000000002,CUST_BOT_OPER,THEME2,None,Андрей,1393,817,55,19,4,32
3,2021-07-01,0000000003,CUST_BOT,THEME2,None,None,726,0,16,8,8,0
4,2021-07-01,0000000004,CUST_BOT,THEME1,None,None,620,0,2,1,1,0


In [16]:
df = df.sort_values('date')

In [17]:
df.to_csv("../data/input/example.csv", index=False)

### В дальнейшем мы сможем выделить все эти особенности с помощью графиков графиков. 